In [ ]:
import glob
import pickle

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib inline

In [ ]:
images = glob.glob('test_images/test*.jpg')

transform_pickle_path = 'wide_transform_pickle.p'
with open(transform_pickle_path, mode='rb') as f:
    transform_pickle = pickle.load(f)
M, Minv = transform_pickle['M'], transform_pickle['Minv']

dist_pickle_path = 'wide_dist_pickle.p'
with open(dist_pickle_path, mode='rb') as f:
    dist_pickle = pickle.load(f)
mtx, dist = dist_pickle['mtx'], dist_pickle['dist']

In [ ]:
# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    binary_output = np.zeros_like(sxbinary)
    binary_output[(s_binary == 1) | (sxbinary == 1)] = 1
    return binary_output

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]


    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images')
    
    # Apply each of the thresholding functions  
    thresholded_img = pipeline(img)
    thresholded_img = cv2.warpPerspective(thresholded_img, M, img_size, flags=cv2.INTER_LINEAR)
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(thresholded_img, cmap='gray')
    if idx == 0:
        sub2.set_title('Original Images with source points')

fig.tight_layout()
plt.show()